# Preparing data

Getting some stats for each restaurant to find a metric to classify between a good and a bad business.

In [1]:
import pandas as pd

In [2]:
restaurants_df = pd.read_csv('restaurants.csv')
waiters_df = pd.read_csv('waiters.csv')
transactions_df = pd.read_csv('transactions.csv')

In [3]:
restaurants_df.head()

,ID,Name,Food Type,Details,Smoking Area,Price Range,City,State,Image
0,1,Ocean's Catch,Mediterranean,"Fresh seafood dishes with a coastal vibe, insp...",Yes,$$$,Houston,TX,pexels-photo-2042591.jpeg
1,2,Spice Symphony,American,An orchestra of flavors from around the world ...,Yes,$$$,San Diego,TX,pexels-photo-29172133.jpeg
2,3,La Bella Cucina,Chinese,"Authentic Italian cuisine served in a cozy, fa...",No,$$$$,Los Angeles,TX,pexels-photo-29132866.jpeg
3,4,The Burger Bar,Mexican,Classic and gourmet burgers with a variety of ...,No,$,Dallas,AZ,pexels-photo-7317354.jpeg
4,5,Veggie Garden,Cafe,"A plant-based paradise featuring vibrant, fres...",Yes,$$$,Los Angeles,TX,pexels-photo-28577205.jpeg


In [4]:
waiters_df.head()

,ID,Name,Restaurant
0,1,Sawyer Gray,1
1,2,Shawn Morris,1
2,3,Robin Brooks,1
3,4,Bailey Adams,2
4,5,Blake Ross,2


In [5]:
transactions_df.head()

,ID,Waiter,People,Total,Tip,Timestamp,Payment Method
0,1,1,10,155.51,16.97,2024-09-09 09:34:00,Debit Card
1,2,1,3,111.61,8.27,2024-09-05 05:05:00,Debit Card
2,3,1,9,191.98,36.99,2024-10-27 02:34:00,Cash
3,4,1,5,123.16,15.39,2024-08-20 19:30:00,Mobile Payment
4,5,1,6,24.14,3.77,2024-08-19 16:09:00,Cash


In [6]:
restaurants_df = restaurants_df.merge(waiters_df["Restaurant"].value_counts().reset_index().rename(columns={"count": "Waiter Count"}), left_on="ID", right_on="Restaurant")
restaurants_df.head()

,ID,Name,Food Type,Details,Smoking Area,Price Range,City,State,Image,Restaurant,Waiter Count
0,1,Ocean's Catch,Mediterranean,"Fresh seafood dishes with a coastal vibe, insp...",Yes,$$$,Houston,TX,pexels-photo-2042591.jpeg,1,3
1,2,Spice Symphony,American,An orchestra of flavors from around the world ...,Yes,$$$,San Diego,TX,pexels-photo-29172133.jpeg,2,6
2,3,La Bella Cucina,Chinese,"Authentic Italian cuisine served in a cozy, fa...",No,$$$$,Los Angeles,TX,pexels-photo-29132866.jpeg,3,10
3,4,The Burger Bar,Mexican,Classic and gourmet burgers with a variety of ...,No,$,Dallas,AZ,pexels-photo-7317354.jpeg,4,4
4,5,Veggie Garden,Cafe,"A plant-based paradise featuring vibrant, fres...",Yes,$$$,Los Angeles,TX,pexels-photo-28577205.jpeg,5,3


In [7]:
transactions_df["Timestamp"] = pd.to_datetime(transactions_df["Timestamp"])
transactions_df.head()

,ID,Waiter,People,Total,Tip,Timestamp,Payment Method
0,1,1,10,155.51,16.97,2024-09-09 09:34:00,Debit Card
1,2,1,3,111.61,8.27,2024-09-05 05:05:00,Debit Card
2,3,1,9,191.98,36.99,2024-10-27 02:34:00,Cash
3,4,1,5,123.16,15.39,2024-08-20 19:30:00,Mobile Payment
4,5,1,6,24.14,3.77,2024-08-19 16:09:00,Cash


In [8]:
transactions_df = transactions_df.merge(waiters_df.rename(columns={"ID": "Waiter"})[["Waiter", "Restaurant"]], left_on="Waiter", right_on="Waiter")
transactions_df.head()

,ID,Waiter,People,Total,Tip,Timestamp,Payment Method,Restaurant
0,1,1,10,155.51,16.97,2024-09-09 09:34:00,Debit Card,1
1,2,1,3,111.61,8.27,2024-09-05 05:05:00,Debit Card,1
2,3,1,9,191.98,36.99,2024-10-27 02:34:00,Cash,1
3,4,1,5,123.16,15.39,2024-08-20 19:30:00,Mobile Payment,1
4,5,1,6,24.14,3.77,2024-08-19 16:09:00,Cash,1


In [9]:
transactions_df["Month"] = transactions_df["Timestamp"].dt.to_period("M").dt.start_time
transactions_df.head()

,ID,Waiter,People,Total,Tip,Timestamp,Payment Method,Restaurant,Month
0,1,1,10,155.51,16.97,2024-09-09 09:34:00,Debit Card,1,2024-09-01
1,2,1,3,111.61,8.27,2024-09-05 05:05:00,Debit Card,1,2024-09-01
2,3,1,9,191.98,36.99,2024-10-27 02:34:00,Cash,1,2024-10-01
3,4,1,5,123.16,15.39,2024-08-20 19:30:00,Mobile Payment,1,2024-08-01
4,5,1,6,24.14,3.77,2024-08-19 16:09:00,Cash,1,2024-08-01


In [10]:
monthly_means = transactions_df.groupby(["Month", "Restaurant"])[["People", "Total", "Tip"]].mean().reset_index().groupby("Restaurant")[["People", "Total", "Tip"]].mean().reset_index()
monthly_means.head()

,Restaurant,People,Total,Tip
0,1,6.119587,110.367585,13.808969
1,2,5.727653,109.592477,14.036327
2,3,5.311244,111.868814,13.980220
3,4,5.325480,108.776504,13.046276
4,5,5.645668,109.974089,13.949825


In [11]:
payment_method_mode = transactions_df.groupby('Restaurant')['Payment Method'].agg(lambda x: x.mode().iloc[0]).reset_index()
payment_method_mode.head()

,Restaurant,Payment Method
0,1,Debit Card
1,2,Cash
2,3,Cash
3,4,Cash
4,5,Credit Card


In [12]:
restaurants_df = restaurants_df.merge(monthly_means, left_on="ID", right_on="Restaurant").merge(payment_method_mode, left_on="ID", right_on="Restaurant")
restaurants_df.head()

,ID,Name,Food Type,Details,Smoking Area,Price Range,City,State,Image,Restaurant_x,Waiter Count,Restaurant_y,People,Total,Tip,Restaurant,Payment Method
0,1,Ocean's Catch,Mediterranean,"Fresh seafood dishes with a coastal vibe, insp...",Yes,$$$,Houston,TX,pexels-photo-2042591.jpeg,1,3,1,6.119587,110.367585,13.808969,1,Debit Card
1,2,Spice Symphony,American,An orchestra of flavors from around the world ...,Yes,$$$,San Diego,TX,pexels-photo-29172133.jpeg,2,6,2,5.727653,109.592477,14.036327,2,Cash
2,3,La Bella Cucina,Chinese,"Authentic Italian cuisine served in a cozy, fa...",No,$$$$,Los Angeles,TX,pexels-photo-29132866.jpeg,3,10,3,5.311244,111.868814,13.980220,3,Cash
3,4,The Burger Bar,Mexican,Classic and gourmet burgers with a variety of ...,No,$,Dallas,AZ,pexels-photo-7317354.jpeg,4,4,4,5.325480,108.776504,13.046276,4,Cash
4,5,Veggie Garden,Cafe,"A plant-based paradise featuring vibrant, fres...",Yes,$$$,Los Angeles,TX,pexels-photo-28577205.jpeg,5,3,5,5.645668,109.974089,13.949825,5,Credit Card


In [13]:
restaurants_df = restaurants_df.drop(columns=["Restaurant", "Restaurant_x", "Restaurant_y"])

In [14]:
import numpy as np

# Adding a column for the classifcation of the restaurants based on the average tip amount per month (good or bad)
def classify_tip(tip, monthly_average):
    if tip > monthly_average:
        return "Good" if np.random.uniform() < 0.9 else "Bad"
    return "Bad" if np.random.uniform() < 0.9 else "Good"

In [15]:
restaurants_df["Classification"] = restaurants_df.apply(lambda row: classify_tip(row["Tip"], restaurants_df["Tip"].mean()), axis=1)
restaurants_df.head()

,ID,Name,Food Type,Details,Smoking Area,Price Range,City,State,Image,Waiter Count,People,Total,Tip,Payment Method,Classification
0,1,Ocean's Catch,Mediterranean,"Fresh seafood dishes with a coastal vibe, insp...",Yes,$$$,Houston,TX,pexels-photo-2042591.jpeg,3,6.119587,110.367585,13.808969,Debit Card,Good
1,2,Spice Symphony,American,An orchestra of flavors from around the world ...,Yes,$$$,San Diego,TX,pexels-photo-29172133.jpeg,6,5.727653,109.592477,14.036327,Cash,Good
2,3,La Bella Cucina,Chinese,"Authentic Italian cuisine served in a cozy, fa...",No,$$$$,Los Angeles,TX,pexels-photo-29132866.jpeg,10,5.311244,111.868814,13.980220,Cash,Good
3,4,The Burger Bar,Mexican,Classic and gourmet burgers with a variety of ...,No,$,Dallas,AZ,pexels-photo-7317354.jpeg,4,5.325480,108.776504,13.046276,Cash,Bad
4,5,Veggie Garden,Cafe,"A plant-based paradise featuring vibrant, fres...",Yes,$$$,Los Angeles,TX,pexels-photo-28577205.jpeg,3,5.645668,109.974089,13.949825,Credit Card,Good


In [16]:
restaurants_df.to_csv('restaurants_with_metrics.csv', index=False)